In [1]:
# Imports 
import gym
import numpy as np


In [2]:
# Define the mountain car environment 
env = gym.make('MountainCar-v0')

/Users/CurtisIrvine/opt/anaconda3/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [3]:
# Observation and action space 
obs_space = env.observation_space
action_space = env.action_space
print(f"The observation space: {obs_space}")
print(f"The action space: {action_space}")

The observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
The action space: Discrete(3)


We now want to define a Q table. This is the quality of every possible action given every possible state. In most cases we will not know the bounds of our observation space. But, this gym enviroment tells us them. 

In [4]:
# Maximum position and momentum values
print(obs_space.high)
# Minimum position and momentum values
print(obs_space.low)

[0.6  0.07]
[-1.2  -0.07]


This observation space is continuous, thus we cannot make a table out of it. We must discretise the observation space to make a Q table.

In [18]:
# Define wanted position observations 
pos_obs = 20
# Also find the position step
dx = (obs_space.high[0] - obs_space.low[0])/pos_obs

# Define wanted momentum observations 
mom_obs = 20 
# Also find the momentum step
dp = (obs_space.high[1] - obs_space.low[1])/mom_obs

# Fill a matrix with all possible position and momentum positions after 
# discretisation
disc_pos = np.arange(obs_space.low[0],
                     obs_space.high[0] + dx,
                     dx)
disc_mom = np.arange(obs_space.low[1],
                     obs_space.high[1] + dp,
                     dp)
# Find the discretised space that is a combination of all of these
disc_space = np.array(np.meshgrid(disc_pos, disc_mom))

# The Q table dimension will be a tensor of this with the action space dimension
# Initialise the Q table with zeros 
q_table = np.zeros((pos_obs,
                    mom_obs,
                    env.action_space.n))

In [ ]:
# A function that will tell us corresponding index of the 
# discrete state we are in given the continous state
def discrete_state_index(
    continuous_state: tuple,
    dx: float,
    dp: float,
):
    disc_state_index = (continuous_state - env.observation_space.low)/[dx, dp]
    return discrete_state_index

In [ ]:
# Now define a function for updating the Q values
# Do this without epsilon initially. May go back to change this 
def new_q_value(
    current_q_table,
    state_index,
    reward,
    learning_rate,
    discount_factor
):